# Ex - GroupBy

### Step 1. Importamos librerías

In [55]:
import pandas as pd
import sqlite3

### Step 2. Importar el conjunto de datos de esta [dirección](https://raw.githubusercontent.com/justmarkham/DAT8/master/data/drinks.csv). 

## PANDAS

In [2]:
drinks = pd.read_csv("https://raw.githubusercontent.com/justmarkham/DAT8/master/data/drinks.csv")

## SQL

In [57]:
#SQL
# Nos conectamos y creamos la db
conexion = sqlite3.connect('Consumo_alcohol.db')

# Escribimos DataFrame en una SQLite tabla
drinks.to_sql('alcohol', conexion, index=False, if_exists='replace')

# Obtenemos un cursor que utilizaremos para hacer las queries
crsr = conexion.cursor()


In [58]:
# Con esta función leemos los datos y lo pasamos a un DataFrame de Pandas
def sql_query(query):

    # Ejecuta la query
    crsr.execute(query)

    # Almacena los datos de la query 
    ans = crsr.fetchall()

    # Obtenemos los nombres de las columnas de la tabla
    names = [description[0] for description in crsr.description]

    return pd.DataFrame(ans,columns=names)

In [59]:
res = crsr.execute("SELECT name FROM sqlite_master WHERE type='table'")
for name in res:
    print(name[0])

alcohol


### Step 3.  Asignarlo a una variable llamada drinks.

In [5]:
drinks.head(10)

,country,beer_servings,spirit_servings,wine_servings,total_litres_of_pure_alcohol,continent
0,Afghanistan,0,0,0,0.0,AS
1,Albania,89,132,54,4.9,EU
2,Algeria,25,0,14,0.7,AF
3,Andorra,245,138,312,12.4,EU
4,Angola,217,57,45,5.9,AF
5,Antigua & Barbuda,102,128,45,4.9,NaN
6,Argentina,193,25,221,8.3,SA
7,Armenia,21,179,11,3.8,EU
8,Australia,261,72,212,10.4,OC
9,Austria,279,75,191,9.7,EU


In [60]:
#SQL
query = '''
SELECT *
FROM alcohol
'''

df = sql_query(query)
df

,country,beer_servings,spirit_servings,wine_servings,total_litres_of_pure_alcohol,continent
0,Afghanistan,0,0,0,0.0,AS
1,Albania,89,132,54,4.9,EU
2,Algeria,25,0,14,0.7,AF
3,Andorra,245,138,312,12.4,EU
4,Angola,217,57,45,5.9,AF
...,...,...,...,...,...,...
188,Venezuela,333,100,3,7.7,SA
189,Vietnam,111,2,1,2.0,AS
190,Yemen,6,0,0,0.1,AS
191,Zambia,32,19,4,2.5,AF


### Step 4. ¿Qué continente bebe más cerveza de media?

In [29]:
continent = drinks.groupby('continent')[['beer_servings']].mean().sort_values(by=['beer_servings'], ascending=False)
continent

,beer_servings
continent,
EU,193.777778
SA,175.083333
OC,89.687500
AF,61.471698
AS,37.045455


In [76]:
#SQL
query = '''
SELECT avg("beer_servings") as media_cerveza, continent
FROM alcohol
GROUP BY continent
ORDER BY media_cerveza DESC
LIMIT 1;
'''

df = sql_query(query)
df

,media_cerveza,continent
0,193.777778,EU


In [31]:
max_beer_continent = continent.head(1).index[0] #-->Uso el index[] porque continent es un indice
max_beer_continent

'EU'

### Step 5. Para cada continente saca las estadísticas de consumo de vino.

In [35]:
continent_wine = drinks.groupby('continent')[['wine_servings']].sum().sort_values(by=['wine_servings'], ascending=False)
continent_wine

,wine_servings
continent,
EU,6400
AF,862
SA,749
OC,570
AS,399


In [77]:
#SQL
query = '''
SELECT SUM("wine_servings") as Consumo_vino, continent
FROM alcohol
GROUP BY continent
ORDER BY Consumo_vino DESC;
'''

df = sql_query(query)
df

,Consumo_vino,continent
0,6400,EU
1,862,AF
2,749,SA
3,570,OC
4,564,None
5,399,AS


### Step 6. Imprimir la media de consumo de alcohol por continente para cada columna

In [37]:
continent_alcohol = drinks.groupby('continent')[['total_litres_of_pure_alcohol']].mean().sort_values(by=['total_litres_of_pure_alcohol'], ascending=False)
continent_alcohol

,total_litres_of_pure_alcohol
continent,
EU,8.617778
SA,6.308333
OC,3.381250
AF,3.007547
AS,2.170455


In [78]:
#SQL
query = '''
SELECT AVG("total_litres_of_pure_alcohol") as Consumo_alcohol, continent
FROM alcohol
GROUP BY continent
ORDER BY Consumo_alcohol DESC;
'''

df = sql_query(query)
df

,Consumo_alcohol,continent
0,8.617778,EU
1,6.308333,SA
2,5.995652,None
3,3.381250,OC
4,3.007547,AF
5,2.170455,AS


### Step 7. Imprimir la mediana del consumo de alcohol por continente para cada columna

In [38]:
median_continent_alcohol = drinks.groupby('continent')[['total_litres_of_pure_alcohol']].median().sort_values(by=['total_litres_of_pure_alcohol'], ascending=False)
median_continent_alcohol

,total_litres_of_pure_alcohol
continent,
EU,10.00
SA,6.85
AF,2.30
OC,1.75
AS,1.20


In [81]:
#SQL --> No admite la funcion MEDIAN(), ni PERCENTILE_CONT(0.5) WITHIN GROUP
query = '''
SELECT AVG("total_litres_of_pure_alcohol") AS Consumo_alcohol, continent
FROM (
    SELECT "total_litres_of_pure_alcohol", continent,
           ROW_NUMBER() OVER (PARTITION BY continent ORDER BY "total_litres_of_pure_alcohol") AS row_num,
           COUNT(*) OVER (PARTITION BY continent) AS total_rows
    FROM alcohol
) AS subquery
WHERE row_num >= (total_rows + 1) / 2 AND row_num <= (total_rows + 2) / 2
GROUP BY continent
ORDER BY Consumo_alcohol DESC;


'''

df = sql_query(query)
df

,Consumo_alcohol,continent
0,10.00,EU
1,6.85,SA
2,6.30,None
3,2.30,AF
4,1.75,OC
5,1.20,AS


### Step 8. Imprime los valores medios, mínimos y máximos del consumo de alcohol por continente.

In [40]:
alcohol_stats = drinks.groupby('continent')['total_litres_of_pure_alcohol'].agg(['mean', 'min', 'max'])
alcohol_stats


,mean,min,max
continent,,,
AF,3.007547,0.0,9.1
AS,2.170455,0.0,11.5
EU,8.617778,0.0,14.4
OC,3.381250,0.0,10.4
SA,6.308333,3.8,8.3


In [82]:
#SQL
query = '''
SELECT AVG("total_litres_of_pure_alcohol") as Consumo_alcohol_medio, 
MAX("total_litres_of_pure_alcohol") as Consumo_alcohol_maximo, 
MIN("total_litres_of_pure_alcohol") as Consumo_alcohol_minimo,
continent

FROM alcohol
GROUP BY continent
ORDER BY Consumo_alcohol_medio DESC;
'''

df = sql_query(query)
df

,Consumo_alcohol_medio,Consumo_alcohol_maximo,Consumo_alcohol_minimo,continent
0,8.617778,14.4,0.0,EU
1,6.308333,8.3,3.8,SA
2,5.995652,11.9,2.2,None
3,3.381250,10.4,0.0,OC
4,3.007547,9.1,0.0,AF
5,2.170455,11.5,0.0,AS


### FORMA LARGA

In [45]:
mean_alcohol_stats = drinks.groupby('continent')['total_litres_of_pure_alcohol'].mean()
max_alcohol_stats = drinks.groupby('continent')['total_litres_of_pure_alcohol'].max()
min_alcohol_stats = drinks.groupby('continent')['total_litres_of_pure_alcohol'].min()

mean_alcohol = pd.DataFrame(mean_alcohol_stats)
min_alcohol = pd.DataFrame(min_alcohol_stats)
max_alcohol = pd.DataFrame(max_alcohol_stats)

In [54]:
stats_alcohol = mean_alcohol.merge(max_alcohol, on='continent')
stats_alcohol = stats_alcohol.merge(min_alcohol, on='continent')
stats_alcohol.rename(columns={"total_litres_of_pure_alcohol_x": "media", 
                            "total_litres_of_pure_alcohol_y": "maximo",
                            "total_litres_of_pure_alcohol": "minimo"})

,media,maximo,minimo
continent,,,
AF,3.007547,9.1,0.0
AS,2.170455,11.5,0.0
EU,8.617778,14.4,0.0
OC,3.381250,10.4,0.0
SA,6.308333,8.3,3.8
